In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
# dates & cleaning
import re
import warnings
warnings.filterwarnings("ignore")
# optional: show full tables in console
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

In [2]:
subsidy_file = "../DATA/amsterdam_subsidies_all_english.xlsx"

In [3]:
subsidy = pd.read_excel(subsidy_file)

In [4]:
subsidy.shape

(56484, 14)

In [5]:
subsidy.head()

,id,case_number,applicant,scheme_name,policy_area,issuing_department,project_name,periodicity_type,amount_requested,amount_granted,grant_decision_publication_date,amount_finalized,finalization_publication_date,subsidy_year
0,0,2023/05049,FC Weesp,Strong Sports Clubs,Sport,Sports and Parks,Versterken van de basis: clubscheidsrechters,One-off,5000.0,3825.0,2024-03-05,3825.0,NaT,2024.0
1,1,2023/03251,Foundation Cliffordstudio,Area-based arts and culture activities Distric...,Culture,District West,West Best Talkshow,One-off,20000.0,20000.0,2023-12-14,20000.0,NaT,2024.0
2,2,2024/00728,Private individual,Subsidy: sustainable Amsterdam vehicles scrappage,Space and sustainability,Space and Sustainability,Scrappage bestelbus,One-off,1500.0,1500.0,2024-03-26,1500.0,NaT,2024.0
3,3,2023/02992,Private individual,Residents' initiatives subsidy scheme Southeast,Social,District Southeast,Subsidy application Burendag Gooioord 2023,One-off,1620.0,1620.0,2023-09-12,1620.0,NaT,2023.0
4,4,2024/00245,BIZ-Association Spuibuurt,Business Investment Zone (BIZ) subsidy scheme,Economy,Economic Affairs,Application BIZ Subsidy 2024 BIZ Spuibuurt,One-off,74816.0,74236.5,2024-02-23,74236.5,2025-12-11,2024.0


In [6]:
subsidy["policy_area"].unique()

<ArrowStringArray>
[                        'Sport',                       'Culture',      'Space and sustainability',
                        'Social',                       'Economy',                     'Education',
             'Social foundation', 'Urban development and housing',                          'Work',
                        'Income',             'Internal Services',        'Governance and support',
                     'Diversity',      'Traffic and public space',    'Services and information\n',
                         'Youth',                          'Care',                    'Operations',
                 'Participation',                       'General',                     'Monuments',
         'Public order & safety']
Length: 22, dtype: str

In [7]:
subsidy= subsidy[subsidy["policy_area"].isin(["Care","Education"])]

In [8]:
subsidy.shape

(4858, 14)

In [9]:
subsidy.head()

,id,case_number,applicant,scheme_name,policy_area,issuing_department,project_name,periodicity_type,amount_requested,amount_granted,grant_decision_publication_date,amount_finalized,finalization_publication_date,subsidy_year
8,42949676431,2017/01865,"""Samen tussen Amstel en IJ"" Foundation voor Op...",Onderwijsvoorzieningen primair Education,Education,"Education, Youth and Care",11e groep nieuwkomersonderwijs,One-off,40000.0,23000.0,NaT,23000.0,NaT,2017.0
19,34359740813,2021/05033,"""Samen tussen Amstel en IJ"" Foundation voor Op...",Budget item subsidy and one-off initiatives OJ...,Education,"Education, Youth and Care",Amsterdam Toelage voor het jaar 2021,One-off,533821.0,533821.0,NaT,533821.0,2023-02-03,2021.0
23,34359740518,2021/04632,"""Samen tussen Amstel en IJ"" Foundation voor Op...",Budget item subsidy and one-off initiatives OJ...,Education,"Education, Youth and Care",Amsterdam Toelage voor het schooljaar 2021-2022,Periodic,533821.0,NaN,NaT,NaN,NaT,2021.0
35,25769805487,2021/04941,"""Samen tussen Amstel en IJ"" Foundation voor Op...","VLOA, vz. Piekaanpak",Education,"Education, Youth and Care",Amsterdamse Familieschool,One-off,NaN,NaN,NaT,NaN,NaT,2021.0
65,42949677602,2023/02558,"""Samen tussen Amstel en IJ"" Foundation voor Op...",Amsterdamse Familie School,Education,"Education, Youth and Care",Amsterdamse Familieschool,One-off,2160000.0,1080000.0,2023-12-14,NaN,NaT,2023.0


In [10]:
subsidy["amount"] = subsidy["amount_finalized"]
subsidy["amount"] = subsidy["amount"].fillna(subsidy["amount_granted"])
subsidy["amount"] = subsidy["amount"].fillna(subsidy["amount_requested"])


In [11]:
subsidy = subsidy[["subsidy_year", "policy_area", "amount"]]



In [12]:
subsidy.head()

,subsidy_year,policy_area,amount
8,2017.0,Education,23000.0
19,2021.0,Education,533821.0
23,2021.0,Education,533821.0
35,2021.0,Education,NaN
65,2023.0,Education,1080000.0


In [15]:
subsidy["subsidy_year"] = pd.to_numeric(subsidy["subsidy_year"], errors="coerce")


In [16]:
subsidy = subsidy.dropna(subset=["subsidy_year"])


In [17]:
subsidy["subsidy_year"] = subsidy["subsidy_year"].astype(int)


In [18]:
subsidy_ts = subsidy.groupby(
    ["subsidy_year", "policy_area"]
)["amount"].sum().reset_index()


In [19]:
subsidy_ts = subsidy_ts.pivot(
    index="subsidy_year",
    columns="policy_area",
    values="amount"
).reset_index()


In [20]:
subsidy_ts = subsidy_ts.sort_values("subsidy_year")


In [21]:
subsidy_ts["education_yoy"] = subsidy_ts["Education"].pct_change()
subsidy_ts["care_yoy"] = subsidy_ts["Care"].pct_change()


In [22]:
subsidy_ts["t"] = range(len(subsidy_ts))


In [23]:
subsidy_ts["edu_lag1"] = subsidy_ts["Education"].shift(1)
subsidy_ts["edu_lag2"] = subsidy_ts["Education"].shift(2)

subsidy_ts["care_lag1"] = subsidy_ts["Care"].shift(1)
subsidy_ts["care_lag2"] = subsidy_ts["Care"].shift(2)


In [24]:
subsidy_ts["edu_roll3"] = subsidy_ts["Education"].rolling(3).mean()
subsidy_ts["care_roll3"] = subsidy_ts["Care"].rolling(3).mean()


In [26]:
subsidy_ts.shape

(12, 12)

In [27]:
subsidy_ts.to_csv("../data_clean/subsidies.csv", index=False)